In [1]:
# !pip install sklearn
# !pip install nltk

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from sklearn.svm import LinearSVC

from nltk.classify import svm
nltk.download('stopwords')

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import gc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Code for creating Naive Bayes Classifier for textual data
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

import string
table = str.maketrans('', '', string.punctuation)

In [4]:
twitter_df = pd.read_csv("data/clean/git_twitter.csv", index_col = "Unnamed: 0")
reddit_df = pd.read_csv("data/clean/reddit.csv", index_col = "Unnamed: 0")
reddit_df = reddit_df.dropna()

In [5]:
twitter_df['Data'] = twitter_df['Data'].str.lower()
reddit_df['Data'] = reddit_df['Data'].str.lower()
twitter_df['Data'] = twitter_df['Data'].apply(lambda x: ' '.join([word.translate(table) for word in x.split()]))
twitter_df['Data'] = twitter_df['Data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
twitter_df['Data'] = twitter_df['Data'].apply(lambda x: ' '.join([porter.stem(word) for word in x.split()]))
reddit_df['Data'] = reddit_df['Data'].apply(lambda x: ' '.join([word.translate(table) for word in x.split()]))
reddit_df['Data'] = reddit_df['Data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
reddit_df['Data'] = reddit_df['Data'].apply(lambda x: ' '.join([porter.stem(word) for word in x.split()]))

In [6]:
merged_df = twitter_df.append(reddit_df, ignore_index="true")

In [7]:
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(sentences)
merged_df = merged_df.dropna()

In [8]:
# (train, test) = train_test_split(merged_df, test_size=0.2, random_state=42, shuffle=True)
# (train, test) = train_test_split(twitter_df, test_size=0.2, random_state=42, shuffle=True)
(train, test) = train_test_split(reddit_df, test_size=0.2, random_state=42, shuffle=True)
(train, val) = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)


In [9]:
train_sentences = train['Data'].to_numpy()
test_sentences = test['Data'].to_numpy()
val_sentences = val['Data'].to_numpy()

train_labels = train['Label'].to_numpy()
test_labels = test['Label'].to_numpy()
val_labels = val['Label'].to_numpy()

In [10]:
# stopwords_set = set(stopwords.words("english"))
# labeled_train_featuresets = []
# all_fine_tuned_words = []

# def remove_stopwords(train_df):
#     for index, row in train_df.iterrows():
#         words_filtered = [e.lower() for e in row.Data.split() if len(e) >= 3]
#         words_without_stopwords = [word for word in words_filtered if not word in stopwords_set]
#         labeled_train_featuresets.append((words_without_stopwords, row.Label))
#         all_fine_tuned_words.extend(words_without_stopwords)
        
# remove_stopwords(train.head(13584))

In [11]:
# def get_word_features(wordlist):
#     wordlist = nltk.FreqDist(wordlist)
#     features = wordlist.keys()
#     return features

# words_features = get_word_features(all_fine_tuned_words)

In [12]:
# def get_word_features(wordlist):
#     wordlist = nltk.FreqDist(wordlist)
#     features = wordlist.keys()
#     return features

# words_features = get_word_features(all_fine_tuned_words)

In [13]:
# def features_func(document):
#     document_words = set(document)
#     features = {}
#     for word in words_features:
#         features['contains(%s)' % word] = (word in document_words)
#     return features

In [14]:
# training_set = nltk.classify.apply_features(features_func,labeled_train_featuresets)
train.head()

,Label,Data
15241,1,cia made cast new super agent three men left p...
4450,0,yeah sometim work well friend mine irl ask pho...
16072,0,game journal custom entitl want desir end product
7235,1,settl conspiraci chat say institut femal privi...
9231,1,realiz your retard


In [15]:
# training_set = nltk.classify.apply_features(features_func,labeled_train_featuresets)
# NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
# NB_classifier.show_most_informative_features(15)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(train['Data'])
test_vectors = vectorizer.transform(test['Data'])


In [17]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, train['Label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Size of train set: ",(len(train.index)), "Size of test set:", (len(test.index)))
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(test['Label'], prediction_linear, output_dict=True)
print('positive: ', report['1'])
print('negative: ', report['0'])
print("-------------")
acc = accuracy_score(test['Label'], prediction_linear)
print(acc)

Size of train set:  10867 Size of test set: 3397
Training time: 14.247190s; Prediction time: 3.897730s
positive:  {'precision': 0.9263657957244655, 'recall': 0.7639569049951028, 'f1-score': 0.8373590982286635, 'support': 1021}
negative:  {'precision': 0.9056751467710372, 'recall': 0.9739057239057239, 'f1-score': 0.9385520178462786, 'support': 2376}
-------------
0.9108036502796585
